# Native scoring using the PREDICT T-SQL function with SQL machine learning

<span style="font-size: 14px;"><a href="https://docs.microsoft.com/en-us/sql/machine-learning/predictions/native-scoring-predict-transact-sql" data-href="https://docs.microsoft.com/en-us/sql/machine-learning/predictions/native-scoring-predict-transact-sql" title="https://docs.microsoft.com/en-us/sql/machine-learning/predictions/native-scoring-predict-transact-sql" is-markdown="true" is-absolute="false">https://docs.microsoft.com/en-us/sql/machine-learning/predictions/native-scoring-predict-transact-sql</a></span>

In [1]:
CREATE DATABASE NativeScoringTest;
GO
USE NativeScoringTest;
GO
DROP TABLE IF EXISTS iris_rx_data;
GO
CREATE TABLE iris_rx_data (
    "Sepal.Length" float not null, "Sepal.Width" float not null
  , "Petal.Length" float not null, "Petal.Width" float not null
  , "Species" varchar(100) null
);
GO

: Msg 1801, Level 16, State 3, Line 1
Database 'NativeScoringTest' already exists. Choose a different database name.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.155

In [2]:
INSERT INTO iris_rx_data ("Sepal.Length", "Sepal.Width", "Petal.Length", "Petal.Width" , "Species")
EXECUTE sp_execute_external_script
  @language = N'R'
  , @script = N'iris_data <- iris;'
  , @input_data_1 = N''
  , @output_data_1_name = N'iris_data';
GO

(150 rows affected)

Total execution time: 00:00:05.996

In [3]:
DROP TABLE IF EXISTS ml_models;
GO
CREATE TABLE ml_models ( model_name nvarchar(100) not null primary key
  , model_version nvarchar(100) not null
  , native_model_object varbinary(max) not null);
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.099

In [4]:
DECLARE @model varbinary(max);
EXECUTE sp_execute_external_script
  @language = N'R'
  , @script = N'
    iris.sub <- c(sample(1:50, 25), sample(51:100, 25), sample(101:150, 25))
    iris.dtree <- rxDTree(Species ~ Sepal.Length + Sepal.Width + Petal.Length + Petal.Width, data = iris[iris.sub, ])
    model <- rxSerializeModel(iris.dtree, realtimeScoringOnly = TRUE)
    '
  , @params = N'@model varbinary(max) OUTPUT'
  , @model = @model OUTPUT
  INSERT [dbo].[ml_models]([model_name], [model_version], [native_model_object])
  VALUES('iris.dtree','v1', @model) ;

STDOUT message(s) from external script: 
Rows Read: 75, Total Rows Processed: 75, Total Chunk Time: 0.022 seconds 
Rows Read: 75, Total Rows Processed: 75, Total Chunk Time: 0.007 seconds 
Rows Read: 75, Total Rows Processed: 75, Total Chunk Time: 0.013 seconds 
Rows Read: 2, Total Rows Processed: 2, Total Chunk Time: 0.001 seconds 
Rows Read: 75, Total Rows Processed: 75, Total Chunk Time: 0.013 seconds 
Rows Read: 2, Total Rows Processed: 2, Total Chunk Time: 0.001 seconds 
Rows Read: 75, Total Rows Processed: 75, Total Chunk Time: 0.003 seconds 
Rows Read: 75, Total Rows Processed: 75, Total Chunk Time: 0.014 seconds 
Rows Read: 2, Total Rows Processed: 2, Total Chunk Time: Less than .001 seconds 
Rows Read: 75, Total Rows Processed: 75, Total Chunk Time: 0.015 seconds 
Rows Read: 2, Total Rows Processed: 2, Total Chunk Time: 0.001 seconds 
Rows Read: 75, Total Rows Processed: 75, Total Chunk Time: Less than .001 seconds

STDOUT message(s) from external script: 
Rows Read: 75, Total Rows Processed: 75, Total Chunk Time: 0.005 seconds 
Rows Read: 75, Total Rows Processed: 75, Total Chunk Time: 0.016 seconds 
Rows Read: 2, Total Rows Processed: 2, Total Chunk Time: 0.001 seconds 
Rows Read: 75, Total Rows Processed: 75, Total Chunk Time: 0.015 seconds 
Rows Read: 2, Total Rows Processed: 2, Total Chunk Time: 0.006 seconds 
Rows Read: 75, Total Rows Processed: 75, Total Chunk Time: 0.001 seconds 
Rows Read: 75, Total Rows Processed: 75, Total Chunk Time: 0.001 seconds 

Elapsed time for DTreeEstimation: 0.415 secs.

Elapsed time for BxDTreeBase: 0.423 secs.

(1 row affected)

Total execution time: 00:00:00.762

In [5]:
SELECT *, datalength(native_model_object)/1024. as model_size_kb
FROM ml_models;

(1 row affected)

Total execution time: 00:00:00.120

model_name model_version native_model_object model_size_kb iris.dtree v1 0x626C6F625C574FBE9004EF710F25A6E4C7BDB27FEC6FF1DDFEA5C5782859172057555304010000003B1400000E472E9C26354AFC8A4474CFD49695C6F3130000000000000000000009000007000000727844547265650005010002000000010B00000389010000010000000000803F0100000000000000010000000000000001000000F4F3F2F100000040060000000000400000000000010000000000000001000000000000000300000000000800000000000000803F00000000CDCC8C3F030000000000000000000000010000000000803F01000000050000007472656531010000000000000001000000F4F3F2F100000040000000000000400000000000010000000000000001000000000000000B00000000034000000000000000803F000000000000803F00000080000000800000000005000000010000000000803F0100000000000000010000000000000001000000F4F3F2F1000000400F0000000000400000000000000000000000000001000000000000000700000000034001000000000000803F000000000000803FFF000000FF00000000000000010000000000803F0100000000000000010000000000000001000000F4F3F2F100000040050000000000400000000000000000000000000001000000000000000800000000014000000000000000803F000000000B010003B5010000010000000000803F0100000000000000010000000000000001000000F4F3F2F100000040060000000000400000000000010000000000000001000000000000000300000000000800000000000000803F00000000CDCC8C3F030000000000000000000000010000000000803F01000000050000007472656531010000000000000001000000F4F3F2F100000040040000000000400000000000050000000000000001000000000000001200000000064000000000000000803F000000000000803FA20700000000F07FA20700000000F07F00000000000000000000F03F0000000000000040000000000000084000000000000018400000000000001C40010000000000803F0100000000000000010000000000000001000000F4F3F2F1000000400F0000000000400000000000000000000000000001000000000000000700000000034001000000000000803F000000000000803FFF000000FF00000000000000010000000000803F0100000000000000010000000000000001000000F4F3F2F100000040050000000000400000000000000000000000000001000000000000000800000000014000000000000000803F000000000B020003D5010000010000000000803F0100000000000000010000000000000001000000F4F3F2F100000040060000000000400000000000010000000000000001000000000000000300000000000800000000000000803F00000000CDCC8C3F030000000000000000000000010000000000803F01000000050000007472656531010000000000000001000000F4F3F2F100000040000000000000400000000000140000000000000001000000000000000B00000000034000000000000000803F000000000000803F0000008000000080000000004B00000019000000320000001900000019000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000020000000000000000000000010000000000803F0100000000000000010000000000000001000000F4F3F2F1000000400F0000000000400000000000000000000000000001000000000000000700000000034001000000000000803F000000000000803FFF000000FF00000000000000010000000000803F0100000000000000010000000000000001000000F4F3F2F100000040050000000000400000000000000000000000000001000000000000000800000000014000000000000000803F000000000B030003B5010000010000000000803F0100000000000000010000000000000001000000F4F3F2F100000040060000000000400000000000010000000000000001000000000000000300000000000800000000000000803F00000000CDCC8C3F030000000000000000000000010000000000803F01000000050000007472656531010000000000000001000000F4F3F2F100000040040000000000400000000000050000000000000001000000000000001200000000064000000000000000803F000000000000803FA20700000000F07FA20700000000F07F00000000000000000000F03F000000000000F03F000000000000004000000000000000400000000000000840010000000000803F0100000000000000010000000000000001000000F4F3F2F1000000400F0000000000400000000000000000000000000001000000000000000700000000034001000000000000803F000000000000803FFF000000FF00000000000000010000000000803F0100000000000000010000000000000001000000F4F3F2F100000040050000000000400000000000000000000000000001000000000000000800000000014000000000000000803F000000000B040003CD020000010000000000803F0100000000000000010000000000000001000000F4F3F2F100000040060000000000400000000000010000000000000001000000000000000300000000000800000000000000803F00000000CDCC8C3F0300000

In [6]:
DECLARE @model varbinary(max) = (
  SELECT native_model_object
  FROM ml_models
  WHERE model_name = 'iris.dtree'
  AND model_version = 'v1');
SELECT d.*, p.*
  FROM PREDICT(MODEL = @model, DATA = dbo.iris_rx_data as d)
  WITH(setosa_Pred float, versicolor_Pred float, virginica_Pred float) as p;
go

(150 rows affected)

Total execution time: 00:00:00.606

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,setosa_Pred,versicolor_Pred,virginica_Pred
5.1,3.5,1.4,0.2,setosa,1,0,0
4.9,3,1.4,0.2,setosa,1,0,0
4.7,3.2,1.3,0.2,setosa,1,0,0
4.6,3.1,1.5,0.2,setosa,1,0,0
5,3.6,1.4,0.2,setosa,1,0,0
5.4,3.9,1.7,0.4,setosa,1,0,0
4.6,3.4,1.4,0.3,setosa,1,0,0
5,3.4,1.5,0.2,setosa,1,0,0
4.4,2.9,1.4,0.2,setosa,1,0,0
4.9,3.1,1.5,0.1,setosa,1,0,0
